In [1]:
!pip install matplotlib
!pip install plotly
!pip install pandas
!pip install dash
!pip install dash-bootstrap-components


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from dash import Dash, html, dash_table, dcc,callback, Output, Input
import dash
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash_bootstrap_components as dbc
import pandas as p
from concurrent.futures import ThreadPoolExecutor
import time

In [3]:
df = pd.read_csv('data_falcon9.csv')

In [10]:
# Initialize the app - incorporate css
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets)

# App layout
app.layout = html.Div([
    
    
    html.Div(className='row', children='Launchsite VS PayloadMass, Launchsite VS Landing Outcome',
             style={'textAlign': 'center', 'color': 'blue', 'fontSize': 30}),
    
    html.Div(className='row', children=[
    dcc.RadioItems(options=['PayloadMass','Class',], 
                   value='PayloadMass', 
                   inline=True,
                    id='my-radio-buttons-final')
    ]),
    
     html.Div(className='row', children=[
        html.Div(className='six columns', children=[
            dash_table.DataTable(data=df.to_dict('records'), page_size=11, style_table={'overflowX': 'auto'})
        ]),
        html.Div(className='six columns', children=[
            dcc.Graph(figure={}, id='histo-chart-final')
        ])
    ])
])

## Add controls to build the interaction
@callback(
    Output(component_id='histo-chart-final', component_property='figure'),
    Input(component_id='my-radio-buttons-final', component_property='value')
)
def update_graph(col_chosen):
    if col_chosen == 'PayloadMass':
        fig1 = px.histogram(df, x='LaunchSite', y=col_chosen, histfunc='sum')
    else:  # Assuming the else case is for 'Class'
        fig1 = px.histogram(df, x='LaunchSite', y=col_chosen, histfunc='sum')
    return fig1

# Run the app
# if __name__ == '__main__':
app.run(debug=True)


In [11]:
app2 = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Orbit selection card
orbit_controls = dbc.Card(
    [
        html.Div(
            [
                dbc.Label("Select Orbit"),
                dcc.Dropdown(
                    id="orbit",
                    options=[{"label": orbit, "value": orbit} for orbit in df["Orbit"].unique()],
                    value=df["Orbit"].unique(),  # Set default to all orbits
                    multi=True,
                ),
            ]
        )
    ]
)



app2.layout = dbc.Container(
    [
        html.H1("Each orbit's PayloadMass on each Launchsite"),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col([orbit_controls], lg=4, sm=12),
                dbc.Col(dcc.Graph(id="graph-launch-data"), lg=8, sm=12),
            ],
            align="center",
        ),
    ]
)

# Update Bar Chart Callback Function
@app2.callback(
    Output("graph-launch-data", "figure"),
    [Input("orbit", "value")]
)
def make_launch_data_graph(selected_orbits):
    dff = df[df["Orbit"].isin(selected_orbits)]
    
    fig = go.Figure()
    
    for site in dff["LaunchSite"].unique():
        dft = dff[dff["LaunchSite"] == site]
        total_payload_mass = dft.groupby("Orbit")["PayloadMass"].sum().reset_index()
        
        fig.add_trace(
            go.Bar(
                x=total_payload_mass["Orbit"],
                y=total_payload_mass["PayloadMass"],
                name=site,
            )
        )

    fig.update_layout(xaxis_tickfont_size=10, barmode="group")
    
    return fig


# if __name__ == "__main__":
#app2.run_server(debug=True)

In [12]:
def run_app1():
    app1.run_server(debug=True, port=8050)

def run_app2():
    app2.run_server(debug=True, port=8051)

if __name__ == '__main__':
    with ThreadPoolExecutor() as executor:
        executor.submit(run_app1)
        time.sleep(2)  
        executor.submit(run_app2)